<a href="https://colab.research.google.com/github/uob-positron-imaging-centre/PEPT-Algorithms-RoPP/blob/main/BirminghamMethod_RoPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank"  href="https://github.com/uob-positron-imaging-centre/pept"><img src="https://github.com/uob-positron-imaging-centre/misc-hosting/blob/master/logo.png?raw=true" style="height:200px; display: block; margin-left: auto; margin-right: auto;"/></a>

# Interactive PEPT Analysis Examples using *PEPT-EM*

> [1] Manger S, Renaud A, Vanneste J. An Expectation-Maximization Algorithm for Positron Emission Particle Tracking https://arxiv.org/abs/2104.07457

---

#### Copyright 2021 the `pept` developers
##### Jupyter Notebook authored by Sam Manger, Andrei Leonard Nicusan and Dr. Kit Windows-Yule for the "PEPT: A Comparative Review" paper, commissioned by the Reports on Progress in Physics journal

Licensed under the GNU License, Version 3.0 (the "License").

---


# 1. Introduction

Positron emission particle tracking (PEPT) is a powerful technique allowing the non-invasive, three-dimensional tracking of one or more radioactive 'tracer' particles through particulate, fluid or multiphase systems. It allows particle or fluid motion to be tracked with sub-millimetre accuracy and sub-millisecond temporal resolution and, due to its use of highly-penetrating 511keV gamma rays, can be used to probe the internal dynamics of even large, dense, optically opaque systems <sup>[[2]](https://www.sciencedirect.com/science/article/pii/016890029390864E) [[3]](https://www.sciencedirect.com/science/article/pii/S0263876208003341) [[4]](https://aip.scitation.org/doi/abs/10.1063/1.4983046@rsi.2017.IMGP2017.issue-1)</sup>. In light of its versatility both in terms of the scales and materials of particles which can be tracked <sup>[[5]](https://www.sciencedirect.com/science/article/pii/S1672251507001455)[[6]](https://www.sciencedirect.com/science/article/pii/S0168900206005341)</sup>, and the sizes and geometries of the systems which can be imaged <sup>[[7]](https://www.sciencedirect.com/science/article/pii/S0168900209001880) [[8]](https://www.sciencedirect.com/science/article/pii/S0029549316000273)</sup> , the technique has wide-ranging applicability in diverse scientific, industrial and biomedical applications.

PEPT is performed by radioactively labelling a particle with a positron-emitting radioisotope such as Fluorine-18 ($^{18}\mathrm{F}$) or Gallium-68 ($^{68}\mathrm{Ga}$), and using the back-to-back gamma rays produced by electron-positron annihilation events in and around the tracer to triangulate its spatial position. Each detected gamma ray represents a **line of response (LoR)** .

## 1.1. This Jupyter Notebook

This interactive Jupyter Notebook illustrates the main processing steps employed by the Birmingham Method<sup>[1]</sup> for radioactive tracer tracking, as described in the Reports on Progress in Physics "PEPT: A Comparative Review" paper.

An [example dataset](https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv) is used from an experiment run at the University of Birmingham Positron Imaging Centre using the ADAC Forté by Matthew Herald. It consists of a single 1 mm diameter MCC particle activated with Fluorine-18 radioactive tracer material inside a bubbling fluidised bed. The fluidised bed was filled with 90% sand and 10% MCC; air was fed into the bottom of the bed at a rate of 37 litres per minute at 3.5 bar. This dataset was chosen for its high quality captured lines of response, with the tracer still depicting the random particle motion that is inherent to bubbling fluidised beds - and typical in Lagrangian particle tracking.

The [`pept`](https://github.com/uob-positron-imaging-centre/pept) Python library is used for initialising and visualising PEPT data. While not required *per se* for illustrating PEPT algorithms' processing steps, it significantly reduces the amount of repetitive code and visual noise, allowing the reader to focus on the main conceptual procedures.

## 1.2. Running Code Cells
Select any code cell and click on the (▶) sign in the top-left of the cell's frame to run its code. 

In [1]:
# First install the `pept` library using pip, Python's package manager
!pip install git+https://github.com/uob-positron-imaging-centre/pept.git

/bin/bash: pip: command not found


# 2. PEPT-EM

## 2.1. Read in Line of Response Data

In [17]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 50,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors_raw = np.insert(lors_raw, 6, head_separation, axis = 1)

# Project the 3D lines onto the YZ plane for ease of analysis - i.e. select only columns
# [time, y1, z1, y2, z2] and flip columns to get [time, x1, y1, x2, y2]
lors = np.array(lors_raw[:, [0, 3, 2, 6, 5]])

# Print the line of response (LoR) data
lors

array([[0.000e+00, 0.000e+00, 1.687e+02, 6.000e+02, 1.428e+02],
       [1.000e-01, 0.000e+00, 1.676e+02, 6.000e+02, 3.139e+02],
       [1.000e-01, 0.000e+00, 4.100e+02, 6.000e+02, 2.401e+02],
       [2.000e-01, 0.000e+00, 2.962e+02, 6.000e+02, 4.525e+02],
       [2.000e-01, 0.000e+00, 1.151e+02, 6.000e+02, 3.534e+02],
       [2.000e-01, 0.000e+00, 1.322e+02, 6.000e+02, 2.661e+02],
       [2.000e-01, 0.000e+00, 3.735e+02, 6.000e+02, 1.310e+02],
       [3.000e-01, 0.000e+00, 1.115e+02, 6.000e+02, 3.534e+02],
       [4.000e-01, 0.000e+00, 2.094e+02, 6.000e+02, 2.808e+02],
       [4.000e-01, 0.000e+00, 2.749e+02, 6.000e+02, 3.062e+02],
       [5.000e-01, 0.000e+00, 2.389e+02, 6.000e+02, 1.681e+02],
       [6.000e-01, 0.000e+00, 4.283e+02, 6.000e+02, 7.020e+01],
       [6.000e-01, 0.000e+00, 1.333e+02, 6.000e+02, 3.499e+02],
       [6.000e-01, 0.000e+00, 3.050e+02, 6.000e+02, 1.032e+02],
       [6.000e-01, 0.000e+00, 9.200e+01, 6.000e+02, 3.929e+02],
       [6.000e-01, 0.000e+00, 1.859e+02,

In [18]:
from pept.visualisation import PlotlyGrapher2D

grapher = PlotlyGrapher2D()
grapher.add_lines(lors)
grapher.show()

## 2.2. Represent the LORs as Vectors

In [19]:
# We recondition our LORs in the form y = x + An

lors_original = np.copy(lors)

#lors = np.copy(lors_original)

lors[:,3] = lors[:,3] - lors[:,1]
lors[:,4] = lors[:,4] - lors[:,2]

lors[:,3:] /= np.linalg.norm(lors[:,3:], axis=-1)[:,np.newaxis]

# And calculate an initial guess for the location (basically the minimum distance point) for these LORs

weights = np.ones(lors.shape[0])
m = np.identity(2)[np.newaxis,:,:] - (lors[:,np.newaxis,3:]*lors[:,3:,np.newaxis])
M = np.sum(m, axis=0)
V = np.sum(np.sum(m*lors[:,np.newaxis,1:3],axis=-1),axis=0)

x = np.matmul(V, np.linalg.inv(M))

X = x[np.newaxis,:2]-lors[:,1:3]     
d2 = np.sum(X**2,axis=-1)-np.sum(X*lors[:,np.newaxis,3:],axis=-1)**2

# Insert MDP timestamp at column 0
x = np.insert(x, 0, lors[:, 0].mean())

print(f"X = {x}")
print(f"d2 = {d2}")

s = np.sqrt(np.mean(d2))

print(s)

X = [  0.938      296.54123338 243.38995732]
d2 = [[7.64037774e+03 7.83370693e+03 2.36133148e+04 ... 8.15527409e+03
  1.03014656e+04 3.23723223e+04]
 [5.36145041e+00 1.14526064e+01 5.38778867e+04 ... 2.63470155e+01
  2.56452529e+02 8.51554211e+03]
 [2.33045342e+04 2.36287969e+04 6.32233282e+03 ... 2.41642412e+04
  2.76306863e+04 5.84455068e+04]
 ...
 [1.01249082e+01 1.80743131e+01 5.35508907e+04 ... 3.60162645e+01
  2.85273130e+02 8.68772711e+03]
 [6.52846625e+01 4.92899183e+01 5.78080529e+04 ... 2.79567859e+01
  3.02204446e+01 6.57505553e+03]
 [1.25581019e+04 1.23585263e+04 9.49750557e+04 ... 1.20353962e+04
  1.01209878e+04 1.35438799e+03]]
139.5487209931238


In [57]:
# Plot all LoRs and the minimum distance point (MDP)
grapher = PlotlyGrapher2D()

grapher.add_lines(lors_original)
grapher.add_points([x], size=s)

grapher.show()